## Tests statistiques : 

On va dans ce notebook réaliser un permutation test pour savoir si nos modèles sont meilleurs que le hasard.

* Objectif : Évaluer la significativité statistique des performances du modèle (AUC).
* Méthode :
    * Réarranger aléatoirement les étiquettes des classes (pathologie vs contrôle sain).
    * Répéter ce réarrangement plusieurs milliers de fois pour créer une distribution nulle des AUC.
* Comparaison :
    *Comparer l'AUC observée du modèle avec la distribution nulle générée.
*Interprétation de la p-value :
    * p-value faible (< 0,05) : Les performances du modèle sont statistiquement significatives et supérieures au hasard.
    * p-value élevée : Les performances du modèle ne sont pas significativement meilleures que celles attendues par le hasard.

### Fonctions auxiliaires 

On va utilise une fonction qui extraie les métriques, les bandes et les pathologies. On va aussi récupérer les "meilleures" bande selon l'AUC (on n'a pas réalisé de test statistique pour savoir si c'est vraiment la meilleure, mais par simplicité on procèdera ainsi). 

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve,auc
from sklearn.ensemble import RandomForestClassifier
import warnings 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

def extract_best_auc(df,disorder):
    df_disorder=df[df["main.disorder"]==disorder]
    df_max=df_disorder.loc[df_disorder["Mean AUC"].idxmax()]
    return(df_max)

def select_pathology_and_metrics(df,colonne,disorder,band,metric):
    variables_categoriques=["sex","education","age","IQ"]
    
    
    df_disorder=df.loc[(df[colonne]==disorder) | (df[colonne]=="healthy control")]
    df_disorder[colonne]=df_disorder[colonne].map({"healthy control" : 0, disorder : 1 } )

    selected_features = [col for col in df_disorder.columns if (band in col.split(sep=".") and metric in (col.split(sep=".")))]
    

    all_elements_present = all(element in df.columns for element in variables_categoriques)
    if all_elements_present:
        selected_features = variables_categoriques+[col for col in df_disorder.columns if (band in col.split(sep=".") and metric in (col.split(sep=".")))]
    else : 
        selected_features = [col for col in df_disorder.columns if (band in col.split(sep=".") and metric in (col.split(sep=".")))]


    y=df_disorder["main.disorder"]
    return df_disorder[selected_features],y



# Réalistion des tests : 

Pour des raisons de temps de calcul on ne réalisera pas plus de 100 permutations.

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
import warnings

warnings.filterwarnings("ignore")

def randomized_labels_nested_cv(
    X, y, 
    model,               # Modèle scikit-learn pré-configuré (sans Grid Search)
    param_grid=None,     # Dictionnaire pour la Grid Search (ex: {'C': [0.1, 0.5, 1, 5, 10]}) ou None
    n_permutations=1000, # Nombre de permutations
    n_splits=5,          # Nombre de plis pour la validation croisée externe
    internal_cv=3,       # Nombre de plis pour la Grid Search interne
    random_state=None    # Graine aléatoire pour reproductibilité
):
    """
    Test de permutation avec validation croisée imbriquée pour évaluer la significativité de l'AUC observée.
    Optionnellement, une Grid Search peut être effectuée pour optimiser les hyperparamètres.
    
    Parameters
    ----------
    X : array-like, shape (n_samples, n_features)
        Données d'entrée.
    y : array-like, shape (n_samples,)
        Étiquettes binaires (0/1).
    model : scikit-learn estimator
        Modèle avec des méthodes `fit` et `predict_proba`.
    param_grid : dict or None, optional
        Grille de recherche pour la Grid Search (ex: {'C': [0.1, 0.5, 1, 5, 10]}) ou None.
    n_permutations : int, default=1000
        Nombre de permutations pour générer la distribution nulle.
    n_splits : int, default=5
        Nombre de plis pour la validation croisée externe.
    internal_cv : int, default=3
        Nombre de plis pour la Grid Search interne.
    random_state : int or None, default=None
        Graine aléatoire pour la reproductibilité.
    
    Returns
    -------
    observed_auc_mean : float
        L'AUC moyenne calculée avec les étiquettes originales.
    p_value : float
        La p-value du test de permutation.
    null_distribution : list
        Distribution nulle des AUC calculées avec des étiquettes randomisées.
    """
    if random_state is not None:
        np.random.seed(random_state)

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # 1) Calculer l'AUC OBSERVÉE via validation croisée externe
    observed_aucs = []
    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Cloner le modèle pour éviter de modifier l'original
        model_clone = clone(model)

        # Si param_grid est fourni, effectuer une Grid Search
        if param_grid is not None:
            grid_search = GridSearchCV(
                estimator=clone(model_clone),
                param_grid=param_grid,
                scoring='roc_auc',
                cv=internal_cv,
                n_jobs=-1
            )
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
        else:
            # Entraîner le modèle directement
            best_model = model_clone.fit(X_train, y_train)

        # Prédictions de probabilité
        try:
            y_proba = best_model.predict_proba(X_test)[:, 1]
        except AttributeError:
            raise AttributeError(f"Le modèle {best_model.__class__.__name__} ne supporte pas 'predict_proba'.")

        # Calcul de l'AUC
        auc = roc_auc_score(y_test, y_proba)
        observed_aucs.append(auc)
        print(f"Fold {fold}: Observed AUC = {auc:.3f}")

    observed_auc_mean = np.mean(observed_aucs)
    print(f"Observed AUC Mean: {observed_auc_mean:.3f}")

    # 2) Générer la distribution nulle via permutations
    null_distribution = []
    for perm in range(1, n_permutations + 1):
        # Permuter les étiquettes
        y_permuted = np.random.permutation(y)

        permuted_aucs = []
        for fold, (train_idx, test_idx) in enumerate(skf.split(X, y_permuted), 1):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train_perm, y_test_perm = y_permuted[train_idx], y[test_idx]

            # Cloner le modèle pour chaque permutation
            model_clone_perm = clone(model)

            # Si param_grid est fourni, effectuer une Grid Search
            if param_grid is not None:
                grid_search_perm = GridSearchCV(
                    estimator=clone(model_clone_perm),
                    param_grid=param_grid,
                    scoring='roc_auc',
                    cv=internal_cv,
                    n_jobs=-1
                )
                grid_search_perm.fit(X_train, y_train_perm)
                best_model_perm = grid_search_perm.best_estimator_
            else:
                # Entraîner le modèle directement
                best_model_perm = model_clone_perm.fit(X_train, y_train_perm)

            # Prédictions de probabilité
            try:
                y_proba_perm = best_model_perm.predict_proba(X_test)[:, 1]
            except AttributeError:
                raise AttributeError(f"Le modèle {best_model_perm.__class__.__name__} ne supporte pas 'predict_proba'.")

            # Calcul de l'AUC
            auc_perm = roc_auc_score(y_test_perm, y_proba_perm)
            permuted_aucs.append(auc_perm)

        # Moyenne des AUC permutées pour cette permutation
        mean_auc_perm = np.mean(permuted_aucs)
        null_distribution.append(mean_auc_perm)

        # Afficher la progression
        if perm % 10 == 0 or perm == 1 or perm == n_permutations:
            print(f"Permutation {perm}/{n_permutations}: Mean Permuted AUC = {mean_auc_perm:.3f}")

    # 3) Calculer la p-value
    null_distribution = np.array(null_distribution)
    p_value = (np.sum(null_distribution >= observed_auc_mean) + 1) / (n_permutations + 1)  # Correction de continuité
    print(f"Mean of null distribution: {null_distribution.mean():.3f}")
    print(f"P-value: {p_value:.4f}")

    return observed_auc_mean, p_value, null_distribution.tolist()

In [14]:
# Charger les données : 

df=pd.read_csv("data\processed\processed_data.csv")
df=df.drop(columns="Unnamed: 0") # retirer la colonne rajoutée lors de la création du csv
df["sex"]=df["sex"].map(lambda x : 0 if x=="M" else 1 ) #Encoder le sexe 
df_svm=pd.read_csv(r"data\processed\models\results_svm.csv")
df_rf=pd.read_csv(r"data\processed\models\results_random_forest.csv")
df_en=pd.read_csv(r"data\processed\models\results_elastic_net.csv")
disorders = [pathologie for pathologie in df_svm["main.disorder"].unique() if pathologie != "healthy control"]



### Random Forest : 

In [72]:
# Identifier la meilleure combinaison (bande + métrique) après validation croisée
results_permutation_rf=[]
for disorder in disorders : 
    df_pathology=df_rf.loc[df_rf["main.disorder"]==disorder]
    best_combination = df_pathology.loc[df_pathology['Mean AUC'].astype(float).idxmax()]
    #best_combination = results_rf_df.loc[results_en_df['Mean AUC Lower CI'].astype(float).idxmax()]
    best_band = best_combination['Band']
    best_metric = best_combination['Metric']
    best_auc = float(best_combination['Mean AUC'])  # Extraire la valeur de l'AUC moyenne

    print(f"Best Model: Disorder={disorder}, Band={best_band}, Metric={best_metric}, AUC={best_auc:.2f}")

    # Extraire les données correspondant à la meilleure combinaison
    X_df, y_s = select_pathology_and_metrics(df, "main.disorder", disorder, best_band, best_metric)
    print(y_s.value_counts())
    X = X_df.values
    y = y_s.values

    # Initialiser le modèle Logistic Regression avec Elastic Net
    rf_model = RandomForestClassifier(n_estimators=500, random_state=42)

    # Effectuer le test de permutation
    observed_auc, p_value, null_distribution = randomized_labels_nested_cv(X, y, rf_model, n_permutations=50,n_splits=5)

    # Calculer l'intervalle de confiance de la distribution nulle
    auc_ci_lower, auc_ci_upper = np.percentile(null_distribution, [2.5, 97.5])

    print(f"Observed AUC: {observed_auc:.3f}")
    print(f"p-value: {p_value:.3f}")
    print(f"95% CI for null distribution: [{auc_ci_lower:.3f}, {auc_ci_upper:.3f}]")

    # Stocker les résultats
    results_permutation_rf.append({
        "Pathology": disorder,
        "Best Band": best_band,
        "Best Metric": best_metric,
        "Observed AUC": observed_auc,
        "p-value": p_value
    })

results_permutation_rf_df = pd.DataFrame(results_permutation_rf)
#results_permutation_df.to_csv("permutation_test_results_en.csv", index=False)
results_permutation_rf_df.to_csv("permutation_test_results_rf.csv",index=False)

Best Model: Disorder=addictive disorder, Band=theta, Metric=COH, AUC=0.80
main.disorder
1    162
0     93
Name: count, dtype: int64
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244


In [3]:
results_permutation_rf_df

,Pathology,Best Band,Best Metric,Observed AUC,p-value
0,addictive disorder,theta,COH,0.779094,0.019608
1,trauma and stress related disorder,beta,COH,0.877982,0.019608
2,mood disorder,beta,COH,0.804444,0.019608
3,obsessive compulsive disorder,alpha,AB,0.740810,0.019608
4,schizophrenia,delta,AB,0.895788,0.019608
5,anxiety disorder,gamma,AB,0.871345,0.019608


On remarque que les p-values sont toutes plus petites que 0.05 ce qui suggère que les modèles sont meilleurs que le hasard. Le fait qu'elles soient constantes et peut être du au faible nombre de permutation combiné à la performance élevée du modèle.

### Elastic Net : 

In [73]:
from sklearn.linear_model import LogisticRegressionCV


results_permutation_en=[]
for disorder in disorders : 
    df_disorder=df_en.loc[df_en["Pathology"]==disorder]
    best_combination = df_disorder.loc[df_disorder['Mean AUC'].astype(float).idxmax()]
    best_band = best_combination['Band']
    best_metric = best_combination['Metric']
    best_auc = float(best_combination['Mean AUC'])  # Extraire la valeur de l'AUC moyenne

    print(f"Best Model: Disorder={disorder}, Band={best_band}, Metric={best_metric}, AUC={best_auc:.2f}")

    # Extraire les données correspondant à la meilleure combinaison
    X_df, y_s = select_pathology_and_metrics(df, "main.disorder", disorder, best_band, best_metric)
    print(X_df)
    X = X_df.values
    y = y_s.values

    # Initialiser le modèle Logistic Regression avec Elastic Net
    logistic = LogisticRegressionCV(
        penalty='elasticnet',
        solver='saga',
        l1_ratios=[0.5],  # Fixing alpha=0.5
        cv=5,             # Internal CV for hyperparameter tuning
        scoring='roc_auc',
        max_iter=10000,
        n_jobs=-1
    )

    # Effectuer le test de permutation
    observed_auc, p_value, null_distribution = randomized_labels_nested_cv(X, y, logistic, n_permutations=50,n_splits=5)

    # Calculer l'intervalle de confiance de la distribution nulle
    auc_ci_lower, auc_ci_upper = np.percentile(null_distribution, [2.5, 97.5])

    print(f"Observed AUC: {observed_auc:.3f}")
    print(f"p-value: {p_value:.3f}")
    print(f"95% CI for null distribution: [{auc_ci_lower:.3f}, {auc_ci_upper:.3f}]")

    # Stocker les résultats
    results_permutation_en.append({
        "Pathology": "Addictive disorder",
        "Best Band": best_band,
        "Best Metric": best_metric,
        
        "Observed AUC": observed_auc,
        "p-value": p_value
    })

results_permutation_en_df = pd.DataFrame(results_permutation_en)
results_permutation_en_df.to_csv("permutation_test_results_en.csv", index=False)
    

Best Model: Disorder=addictive disorder, Band=beta, Metric=AB, AUC=0.79
     sex  education   age     IQ  AB.D.beta.a.FP1  AB.D.beta.b.FP2  \
0      1        6.0  37.0  120.0        30.423142        23.859927   
1      1       16.0  32.0  113.0        13.799929        10.315396   
2      1       18.0  35.0  126.0         6.429691         6.212037   
3      1       16.0  36.0  112.0        10.147936        11.017907   
4      1       14.0  24.0  105.0        12.002400        12.032555   
..   ...        ...   ...    ...              ...              ...   
818    1       13.0  22.0  116.0         9.969296         5.815453   
819    1       13.0  26.0  118.0         8.358988         8.504071   
820    1       16.0  26.0  113.0        16.310102        16.320458   
821    1       13.0  24.0  107.0        11.177623        11.777143   
822    1       13.0  21.0  105.0        11.256001         8.588518   

     AB.D.beta.c.F7  AB.D.beta.d.F3  AB.D.beta.e.Fz  AB.D.beta.f.F4  ...  \
0         2

In [ ]:
results_permutation_en_df

,Pathology,Best Band,Best Metric,Observed AUC,p-value
0,Addictive disorder,beta,AB,0.683597,0.019608
1,Addictive disorder,beta,COH,0.862400,0.019608
2,Addictive disorder,delta,AB,0.782338,0.019608
3,Addictive disorder,beta,COH,0.609696,0.117647
4,Addictive disorder,beta,AB,0.865480,0.019608
5,Addictive disorder,theta,COH,0.794152,0.019608


On observe encore une fois des p-values inférieures à 0.05 sauf pour les troubles obsessionnels compulsifs, cela peut s'expliquer par le faible nombre de permutations et la moins bonne performance du modèle sur cette pathologie.  

In [ ]:
from sklearn.svm import SVC

# Identifier la meilleure combinaison (bande + métrique) après validation croisée
C_values={"C" : [0.1,0.5,1,5,10]}
results_permutation_svm=[]
for pathology in disorders:
    print(pathology)
    df_pathology=df_svm.loc[df_svm["main.disorder"]==pathology]
    
    best_combination = df_pathology.loc[df_pathology['Mean AUC'].astype(float).idxmax()]
    
    best_band = best_combination['Band']
    best_metric = best_combination['Metric']
    best_auc = float(best_combination['Mean AUC'])

    print(f"Best Model: Disorder={pathology}, Band={best_band}, Metric={best_metric}, AUC={best_auc:.2f}")

    # Extraire les données correspondant à la meilleure combinaison
    X, y = select_pathology_and_metrics(df, "main.disorder", pathology, best_band, best_metric)

    X, y = X.values, y.values

    # Utiliser la meilleure valeur de C
    #best_C = C_values[np.argmax([float(df_svm.loc[df_svm['Band'] == best_band]['Mean AUC'].values[0])])]
    svm_best = SVC(kernel='rbf', probability=True)

    # Effectuer le test de permutation
    observed_auc, p_value, null_distribution = randomized_labels_nested_cv(X, y, svm_best, n_splits=10,n_permutations=100,param_grid=C_values)
   

    # Calculer l'intervalle de confiance de la distribution nulle
    auc_ci_lower, auc_ci_upper = np.percentile(null_distribution, [2.5, 97.5])

    print(f"Observed AUC: {observed_auc:.3f}")
    print(f"p-value: {p_value:.6f}")
    print(f"95% CI for null distribution: [{auc_ci_lower:.3f}, {auc_ci_upper:.3f}]")

    # Stocker les résultats
    results_permutation_svm.append({
        " Disorder": f"{pathology}",
        "Best Band": best_band,
        "Best Metric": best_metric,
        "Observed AUC": observed_auc,
        "Best AUC" : best_auc,
        
        "p-value": p_value
    })

results_permutation_svm_df = pd.DataFrame(results_permutation_svm)
results_permutation_svm_df.to_csv("permutation_test_results_svm_fixed_params.csv", index=False)
    

In [24]:
results_permutation_svm_df

,Disorder,Best Band,Best Metric,Observed AUC,Best AUC,p-value
0,addictive disorder,delta,COH,0.752380,0.79,0.009901
1,trauma and stress related disorder,gamma,AB,0.884058,0.89,0.009901
2,mood disorder,gamma,AB,0.816076,0.84,0.009901
3,obsessive compulsive disorder,highbeta,COH,0.693254,0.74,0.009901
4,schizophrenia,delta,AB,0.915024,0.91,0.009901
5,anxiety disorder,delta,AB,0.824561,0.86,0.009901


Encore une fois on peut observer des p-values faibles ce qui signifie que nos modèles sont très probablement meilleurs que le hasard.